<h1><center>P7 - CREATE & IMPLEMENT A CREDIT SCORING MODEL</center></h1>

<div style="width:100%;text-align: center;"> <img align=middle src="http://www.financialbuzz.co.uk/wp-content/uploads/2012/04/Loans-For-Bad-Credit.jpg" alt="Heat beating" style="height:400px;margin-top:3rem;"> </div>

## Part 4 - Data preparation for Dashboard

## Table of Contents

* [1. Dataset Importation](#chapter1)
    * [1.1 Full model](#section_1_1)
    * [1.2 Selected Features model (via BorutaPy)](#section_1_2)
* [2. Models comparison](#chapter2)
    * [2.1 Overview](#section_2_1)
    * [2.2 LightGBM](#section_2_2)
    * [2.3 XGBoost](#section_2_3)
    * [2.4 CatBoost](#section_2_4)

In [62]:
# ! usr/bin/env python 3
# coding: utf-8

import datetime
import requests
import json
import sys
import time
from datetime import datetime
import warnings
import numpy as np
import pandas as pd
import os
import pickle
from collections import Counter
import warnings

# Visualisation
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import joblib as joblib

# Data pré-processing
import sklearn
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

# Metrics
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, \
                            recall_score, precision_score, matthews_corrcoef, \
                            cohen_kappa_score, fbeta_score, make_scorer, \
                            average_precision_score, log_loss, confusion_matrix, \
                            classification_report
# Interprétation
import shap

# Warnings

from warnings import simplefilter
simplefilter("ignore", category=Warning)
warnings.filterwarnings('ignore')
    
from IPython.core.display import display, HTML

from func_modp7 import*

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
# Versions
print('Version des librairies utilisées :')

print('Python                : ' + sys.version)
print('NumPy                 : ' + np.version.full_version)
print('Pandas                : ' + pd.__version__)
print('Scikit-learn          : ' + sklearn.__version__)
print('Matplotlib            : ' + matplotlib.__version__)
print('Seaborn               : ' + sns.__version__)
print('Shap                 : ' + shap.__version__)
print('Lightgbm              : ' + lgb.__version__)

Version des librairies utilisées :
Python                : 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
NumPy                 : 1.19.5
Pandas                : 1.2.4
Scikit-learn          : 0.23.2
Matplotlib            : 3.3.4
Seaborn               : 0.11.1
Shap                 : 0.40.0
Lightgbm              : 3.3.2


All functions run in this notebook can be found at: functions_P7_feat.py

In [64]:
start_time = datetime.now()

## 1. Dataset importation for dashboard<a class="anchor" id="chapter1"></a>

In [65]:
application_test = pd.read_csv('application_test.csv')
application_test_ltd = application_test.head(2000)
application_test_ltd.to_pickle('./application_test_ltd.pkl')

In [66]:
application_train = pd.read_csv('application_train.csv')
application_train_ltd = application_train.head(2000)
application_train_ltd.to_pickle('./application_train_ltd.pkl')

### 1.1 Selected features model (via BorutaPy)<a class="anchor" id="section_1_2"></a>

In [67]:
# Load data - Lighter range after Boruta
data_train_boruta = pd.read_pickle("train_data_boruta.pkl")
data_test_boruta = pd.read_pickle("test_data_boruta.pkl")
best_model = joblib.load("final_model_w.pkl")
print("Data train shape is:", data_train_boruta.shape)
print("Data test shape is:", data_test_boruta.shape)
best_model

Data train shape is: (307506, 88)
Data test shape is: (48744, 87)


LGBMClassifier(bagging_fraction=0.6000000000000001, bagging_freq=1,
               feature_fraction=0.9, lambda_l1=40, lambda_l2=20,
               learning_rate=0.12624129923876112, max_depth=12,
               min_data_in_leaf=200, min_gain_to_split=0.4518795605436023,
               n_estimators=10000, num_leaves=1180)

In [68]:
# Limited dataset for deployment into Heroku due to memory errors
data_test_ltd = data_test_boruta.head(2000)
data_test_ltd

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,...,INS_DBD_MEAN,INS_LATE_PAYMENT_SUM,INS_36M_DAYS_ENTRY_PAYMENT_MIN,INS_36M_AMT_INSTALMENT_MEAN,INS_36M_AMT_PAYMENT_SUM,INS_36M_SIGNIFICANT_LATE_PAYMENT_MEAN,INS_60M_LATE_PAYMENT_MEAN,INS_60M_SIGNIFICANT_LATE_PAYMENT_MEAN,BUREAU_ACTIVE_CREDIT_TO_INCOME_RATIO,CURRENT_TO_APPROVED_CREDIT_MIN_RATIO
307506,100001,0,0,135000.0,568800.0,0,0,1,0,0.018845,...,8.859375,4.0,-840.0,15708.870117,3.499691e+05,0.910645,1.000000,1.000000,6.546875,0.041809
307507,100005,0,0,99000.0,222768.0,0,0,1,0,0.035797,...,23.671875,8.0,-736.0,6240.205078,5.616184e+04,0.888672,0.888672,0.888672,6.046875,0.180298
307508,100013,0,1,202500.0,663264.0,-1,0,1,0,0.019104,...,5.722656,49.0,-2704.0,11124.233398,1.444700e+06,0.226074,0.316162,0.270996,4.632812,0.036530
307509,100028,0,0,315000.0,1575000.0,0,0,1,0,0.026398,...,3.265625,55.0,-1302.0,4730.878418,3.506999e+05,0.348389,0.486816,0.486816,2.746094,0.037781
307510,100038,0,1,180000.0,625500.0,0,0,1,0,0.010033,...,12.250000,12.0,-802.0,11100.337891,1.332040e+05,1.000000,1.000000,1.000000,4.632812,0.148071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309501,114010,0,0,90000.0,1161360.0,0,3,1,0,0.072510,...,11.695312,13.0,-846.0,51262.222656,8.351589e+05,1.000000,1.000000,1.000000,4.632812,0.166138
309502,114011,0,0,180000.0,286704.0,0,1,1,0,0.007114,...,12.179688,40.0,-1295.0,35488.746094,9.581962e+05,0.852051,0.909180,0.909180,5.710938,0.116211
309503,114032,0,1,225000.0,1528200.0,0,0,1,0,0.019104,...,12.906250,11.0,-840.0,15708.870117,3.499691e+05,0.910645,0.909180,0.902344,3.160156,0.051422
309504,114047,0,0,99000.0,350415.0,1,3,2,0,0.031342,...,9.976562,46.0,-1233.0,15319.239258,6.177781e+05,0.975586,0.978516,0.978516,4.632812,0.059662


In [69]:
# Limited dataset for deployment into Heroku due to memory errors
data_train_ltd = data_train_boruta.head(2000)
data_train_ltd.drop('TARGET', axis=1, inplace=True)

Information bbout the customer to be displayed:

**DAYS_BIRTH**: Age - ok but years instead of days

**CODE_GENDER**: Sexe - To add

**NAME_FAMILY_STATUS**: Family Statut - To add (encoded)

**CNT_CHILDREN**: Number of kids - To add

**NAME_EDUCATION_TYPE**: Education Type - To add

**NAME_INCOME_TYPE**: Income type - To add (encoded)

**DAYS_EMPLOYED**: ok but days in years 

**AMT_INCOME_TOTAL**: Total Income - OK

About the loan itself:

**NAME_CONTRACT_TYPE**: Loan type - Replace 0 by cash loans and 1 by revolving loans

**AMT_CREDIT**: Loan total amount - OK

**AMT_ANNUITY**: Annuity - To add

**AMT_GOODS_PRICE**: Goods total price - To add

**NAME_HOUSING_TYPE**: Housing type - To add (encoded)

In [70]:
df_missing = application_test[['SK_ID_CURR', 'CODE_GENDER', 'NAME_FAMILY_STATUS',
                               'CNT_CHILDREN', 'NAME_EDUCATION_TYPE',
                               'NAME_INCOME_TYPE', 
                               'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_HOUSING_TYPE']]
df_missing.head(3)

,SK_ID_CURR,CODE_GENDER,NAME_FAMILY_STATUS,CNT_CHILDREN,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_HOUSING_TYPE
0,100001,F,Married,0,Higher education,Working,20560.5,450000.0,House / apartment
1,100005,M,Married,0,Secondary / secondary special,Working,17370.0,180000.0,House / apartment
2,100013,M,Married,0,Higher education,Working,69777.0,630000.0,House / apartment


In [71]:
df_missing_tr = application_train[['SK_ID_CURR', 'CODE_GENDER', 'NAME_FAMILY_STATUS',
                               'CNT_CHILDREN', 'NAME_EDUCATION_TYPE',
                               'NAME_INCOME_TYPE', 
                               'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_HOUSING_TYPE']]
df_missing_tr.head(3)

,SK_ID_CURR,CODE_GENDER,NAME_FAMILY_STATUS,CNT_CHILDREN,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_HOUSING_TYPE
0,100002,M,Single / not married,0,Secondary / secondary special,Working,24700.5,351000.0,House / apartment
1,100003,F,Married,0,Higher education,State servant,35698.5,1129500.0,House / apartment
2,100004,M,Single / not married,0,Secondary / secondary special,Working,6750.0,135000.0,House / apartment


In [72]:
dash_df = data_test_boruta.merge(df_missing, on='SK_ID_CURR', how='right')
dash_df.head(3)

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,NAME_INCOME_TYPE_x,NAME_FAMILY_STATUS_x,NAME_HOUSING_TYPE_x,REGION_POPULATION_RELATIVE,...,BUREAU_ACTIVE_CREDIT_TO_INCOME_RATIO,CURRENT_TO_APPROVED_CREDIT_MIN_RATIO,CODE_GENDER,NAME_FAMILY_STATUS_y,CNT_CHILDREN,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE_y,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_HOUSING_TYPE_y
0,100001,0,0,135000.0,568800.0,0,0,1,0,0.018845,...,6.546875,0.041809,F,Married,0,Higher education,Working,20560.5,450000.0,House / apartment
1,100005,0,0,99000.0,222768.0,0,0,1,0,0.035797,...,6.046875,0.180298,M,Married,0,Secondary / secondary special,Working,17370.0,180000.0,House / apartment
2,100013,0,1,202500.0,663264.0,-1,0,1,0,0.019104,...,4.632812,0.036530,M,Married,0,Higher education,Working,69777.0,630000.0,House / apartment


In [73]:
dashboard = data_train_boruta.merge(df_missing_tr, on='SK_ID_CURR', how='right')
dashboard.drop('TARGET', axis=1, inplace=True)
dashboard = dashboard.head(2000)
dashboard

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,NAME_INCOME_TYPE_x,NAME_FAMILY_STATUS_x,NAME_HOUSING_TYPE_x,REGION_POPULATION_RELATIVE,...,BUREAU_ACTIVE_CREDIT_TO_INCOME_RATIO,CURRENT_TO_APPROVED_CREDIT_MIN_RATIO,CODE_GENDER,NAME_FAMILY_STATUS_y,CNT_CHILDREN,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE_y,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_HOUSING_TYPE_y
0,100002,0.0,0.0,202500.0,406597.5,0.0,0.0,0.0,0.0,0.018799,...,2.380859,0.440430,M,Single / not married,0,Secondary / secondary special,Working,24700.5,351000.0,House / apartment
1,100003,0.0,0.0,270000.0,1293502.5,1.0,1.0,1.0,0.0,0.003542,...,3.000000,0.052612,F,Married,0,Higher education,State servant,35698.5,1129500.0,House / apartment
2,100004,1.0,1.0,67500.0,135000.0,0.0,0.0,0.0,0.0,0.010033,...,4.390625,0.148926,M,Single / not married,0,Secondary / secondary special,Working,6750.0,135000.0,House / apartment
3,100006,0.0,0.0,135000.0,312682.5,0.0,0.0,2.0,0.0,0.008018,...,4.390625,0.077454,F,Civil marriage,0,Secondary / secondary special,Working,29686.5,297000.0,House / apartment
4,100007,0.0,0.0,121500.0,513000.0,0.0,0.0,0.0,0.0,0.028656,...,4.390625,0.028488,M,Single / not married,0,Secondary / secondary special,Working,21865.5,513000.0,House / apartment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,102337,0.0,0.0,49500.0,130824.0,0.0,3.0,4.0,0.0,0.010643,...,4.390625,0.571777,F,Separated,0,Secondary / secondary special,Pensioner,5062.5,103500.0,House / apartment
1996,102338,0.0,1.0,135000.0,254700.0,0.0,0.0,1.0,0.0,0.018631,...,4.390625,0.281250,M,Married,0,Secondary / secondary special,Working,17019.0,225000.0,House / apartment
1997,102339,1.0,1.0,135000.0,202500.0,0.0,0.0,1.0,0.0,0.003069,...,4.390625,0.166138,F,Married,1,Higher education,Working,10125.0,202500.0,House / apartment
1998,102341,1.0,0.0,202500.0,337500.0,0.0,0.0,0.0,0.0,0.072510,...,3.189453,0.279785,F,Single / not married,2,Secondary / secondary special,Working,16875.0,337500.0,House / apartment


In [74]:
# Transform DAYS_BIRTH in positive years
dash_df['YEAR_BIRTH'] = \
    np.trunc(np.abs(dash_df['DAYS_BIRTH'] / 365)).astype('int8')
# Transform DAYS_EMPLOYED in positive years
dash_df['YEAR_EMPLOYED'] = \
    np.trunc(np.abs(dash_df['DAYS_EMPLOYED'] / 365)).astype('int8')
# Transform NAME_CONTRACT_TYPE : 0 = Cash Loans et 1 = Revolving Loans
dash_df['NAME_CONTRACT_TYPE'] = ['Cash Loans' if row == 0 else 'Revolving Loans'
                        for row in dash_df['NAME_CONTRACT_TYPE']] 
dash_df.drop(['NAME_INCOME_TYPE_x', 'NAME_FAMILY_STATUS_x', 'NAME_HOUSING_TYPE_x'], axis = 1, inplace=True)
dict = {'NAME_INCOME_TYPE_y': 'NAME_INCOME_TYPE',
        'NAME_FAMILY_STATUS_y': 'NAME_FAMILY_STATUS',
        'NAME_HOUSING_TYPE_y': 'NAME_HOUSING_TYPE'}
 
# call rename () method
dash_df.rename(columns=dict,
          inplace=True)
dash_df

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,...,CODE_GENDER,NAME_FAMILY_STATUS,CNT_CHILDREN,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_HOUSING_TYPE,YEAR_BIRTH,YEAR_EMPLOYED
0,100001,Cash Loans,0,135000.0,568800.0,0,0.018845,-19241,-2328.0,-1,...,F,Married,0,Higher education,Working,20560.5,450000.0,House / apartment,52,6
1,100005,Cash Loans,0,99000.0,222768.0,0,0.035797,-18064,-4468.0,13,...,M,Married,0,Secondary / secondary special,Working,17370.0,180000.0,House / apartment,49,12
2,100013,Cash Loans,1,202500.0,663264.0,-1,0.019104,-20038,-4456.0,4,...,M,Married,0,Higher education,Working,69777.0,630000.0,House / apartment,54,12
3,100028,Cash Loans,0,315000.0,1575000.0,0,0.026398,-13976,-1866.0,5,...,F,Married,2,Secondary / secondary special,Working,49018.5,1575000.0,House / apartment,38,5
4,100038,Cash Loans,1,180000.0,625500.0,0,0.010033,-13040,-2192.0,-1,...,M,Married,1,Secondary / secondary special,Working,32067.0,625500.0,House / apartment,35,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash Loans,0,121500.0,412560.0,0,0.002043,-19970,-5168.0,-1,...,F,Widow,0,Secondary / secondary special,Working,17473.5,270000.0,House / apartment,54,14
48740,456222,Cash Loans,0,157500.0,622413.0,0,0.035797,-11186,-1149.0,5,...,F,Married,2,Secondary / secondary special,Commercial associate,31909.5,495000.0,House / apartment,30,3
48741,456223,Cash Loans,1,202500.0,315000.0,0,0.026398,-15922,-3036.0,-1,...,F,Married,1,Secondary / secondary special,Commercial associate,33205.5,315000.0,House / apartment,43,8
48742,456224,Cash Loans,0,225000.0,450000.0,1,0.018845,-13968,-2732.0,3,...,M,Married,0,Higher education,Commercial associate,25128.0,450000.0,House / apartment,38,7


In [75]:
# Transform DAYS_BIRTH in positive years
dashboard['YEAR_BIRTH'] = \
    np.trunc(np.abs(dashboard['DAYS_BIRTH'] / 365)).astype('int8')
# Transform DAYS_EMPLOYED in positive years
dashboard['YEAR_EMPLOYED'] = \
    np.trunc(np.abs(dashboard['DAYS_EMPLOYED'] / 365)).astype('int8')
# Transform NAME_CONTRACT_TYPE : 0 = Cash Loans et 1 = Revolving Loans
dashboard['NAME_CONTRACT_TYPE'] = ['Cash Loans' if row == 0 else 'Revolving Loans'
                        for row in dashboard['NAME_CONTRACT_TYPE']] 
dashboard.drop(['NAME_INCOME_TYPE_x', 'NAME_FAMILY_STATUS_x', 'NAME_HOUSING_TYPE_x'], axis = 1, inplace=True)
dict = {'NAME_INCOME_TYPE_y': 'NAME_INCOME_TYPE',
        'NAME_FAMILY_STATUS_y': 'NAME_FAMILY_STATUS',
        'NAME_HOUSING_TYPE_y': 'NAME_HOUSING_TYPE'}
 
# call rename () method
dashboard.rename(columns=dict,
          inplace=True)
dashboard

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,...,CODE_GENDER,NAME_FAMILY_STATUS,CNT_CHILDREN,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_HOUSING_TYPE,YEAR_BIRTH,YEAR_EMPLOYED
0,100002,Cash Loans,0.0,202500.0,406597.5,0.0,0.018799,-9461.0,-637.0,0.0,...,M,Single / not married,0,Secondary / secondary special,Working,24700.5,351000.0,House / apartment,25,1
1,100003,Cash Loans,0.0,270000.0,1293502.5,1.0,0.003542,-16765.0,-1188.0,1.0,...,F,Married,0,Higher education,State servant,35698.5,1129500.0,House / apartment,45,3
2,100004,Revolving Loans,1.0,67500.0,135000.0,0.0,0.010033,-19046.0,-225.0,0.0,...,M,Single / not married,0,Secondary / secondary special,Working,6750.0,135000.0,House / apartment,52,0
3,100006,Cash Loans,0.0,135000.0,312682.5,0.0,0.008018,-19005.0,-3040.0,0.0,...,F,Civil marriage,0,Secondary / secondary special,Working,29686.5,297000.0,House / apartment,52,8
4,100007,Cash Loans,0.0,121500.0,513000.0,0.0,0.028656,-19932.0,-3038.0,1.0,...,M,Single / not married,0,Secondary / secondary special,Working,21865.5,513000.0,House / apartment,54,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,102337,Cash Loans,0.0,49500.0,130824.0,0.0,0.010643,-21713.0,-1648.0,-1.0,...,F,Separated,0,Secondary / secondary special,Pensioner,5062.5,103500.0,House / apartment,59,4
1996,102338,Cash Loans,1.0,135000.0,254700.0,0.0,0.018631,-14209.0,-139.0,13.0,...,M,Married,0,Secondary / secondary special,Working,17019.0,225000.0,House / apartment,38,0
1997,102339,Revolving Loans,1.0,135000.0,202500.0,0.0,0.003069,-14744.0,-5752.0,1.0,...,F,Married,1,Higher education,Working,10125.0,202500.0,House / apartment,40,15
1998,102341,Revolving Loans,0.0,202500.0,337500.0,0.0,0.072510,-10100.0,-3084.0,-1.0,...,F,Single / not married,2,Secondary / secondary special,Working,16875.0,337500.0,House / apartment,27,8


#### Gather basic information about the customer

In [76]:
# Dataframe creation 
df_customer_info = dash_df[['SK_ID_CURR', 'YEAR_BIRTH', 'CODE_GENDER',
                               'NAME_FAMILY_STATUS',
                               'CNT_CHILDREN', 'NAME_EDUCATION_TYPE',
                               'NAME_INCOME_TYPE', 'YEAR_EMPLOYED',
                               'AMT_INCOME_TOTAL']]
df_customer_info.head()

,SK_ID_CURR,YEAR_BIRTH,CODE_GENDER,NAME_FAMILY_STATUS,CNT_CHILDREN,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE,YEAR_EMPLOYED,AMT_INCOME_TOTAL
0,100001,52,F,Married,0,Higher education,Working,6,135000.0
1,100005,49,M,Married,0,Secondary / secondary special,Working,12,99000.0
2,100013,54,M,Married,0,Higher education,Working,12,202500.0
3,100028,38,F,Married,2,Secondary / secondary special,Working,5,315000.0
4,100038,35,M,Married,1,Secondary / secondary special,Working,6,180000.0


In [77]:
# Dataframe creation 
df_customer_info_tr = dashboard[['SK_ID_CURR', 'YEAR_BIRTH', 'CODE_GENDER',
                               'NAME_FAMILY_STATUS',
                               'CNT_CHILDREN', 'NAME_EDUCATION_TYPE',
                               'NAME_INCOME_TYPE', 'YEAR_EMPLOYED',
                               'AMT_INCOME_TOTAL']]
df_customer_info_tr.head()

,SK_ID_CURR,YEAR_BIRTH,CODE_GENDER,NAME_FAMILY_STATUS,CNT_CHILDREN,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE,YEAR_EMPLOYED,AMT_INCOME_TOTAL
0,100002,25,M,Single / not married,0,Secondary / secondary special,Working,1,202500.0
1,100003,45,F,Married,0,Higher education,State servant,3,270000.0
2,100004,52,M,Single / not married,0,Secondary / secondary special,Working,0,67500.0
3,100006,52,F,Civil marriage,0,Secondary / secondary special,Working,8,135000.0
4,100007,54,M,Single / not married,0,Secondary / secondary special,Working,8,121500.0


In [78]:
df_customer_info = df_customer_info.rename(columns = {
     'YEAR_BIRTH' : 'Age (years)',
     'CODE_GENDER' : 'Gender',
     'NAME_FAMILY_STATUS' : 'Family Status',
     'CNT_CHILDREN' : 'Kids',
     'NAME_EDUCATION_TYPE' : 'Education',
     'NAME_INCOME_TYPE' : 'Income Type',
     'YEAR_EMPLOYED' : 'Years Employed',
     'AMT_INCOME_TOTAL' : 'Income ($)'})
df_customer_info.loc[df_customer_info['SK_ID_CURR'] == 100271]

,SK_ID_CURR,Age (years),Gender,Family Status,Kids,Education,Income Type,Years Employed,Income ($)
36,100271,63,F,Married,0,Secondary / secondary special,Pensioner,4,270000.0


In [79]:
df_customer_info_tr = df_customer_info_tr.rename(columns = {
     'YEAR_BIRTH' : 'Age (years)',
     'CODE_GENDER' : 'Gender',
     'NAME_FAMILY_STATUS' : 'Family Status',
     'CNT_CHILDREN' : 'Kids',
     'NAME_EDUCATION_TYPE' : 'Education',
     'NAME_INCOME_TYPE' : 'Income Type',
     'YEAR_EMPLOYED' : 'Years Employed',
     'AMT_INCOME_TOTAL' : 'Income ($)'})
df_customer_info_tr.loc[df_customer_info_tr['SK_ID_CURR'] == 100002]

,SK_ID_CURR,Age (years),Gender,Family Status,Kids,Education,Income Type,Years Employed,Income ($)
0,100002,25,M,Single / not married,0,Secondary / secondary special,Working,1,202500.0


In [80]:
df_customer_info.head(2000).to_pickle('./df_customer_info.pkl')

In [81]:
df_customer_info_tr.to_pickle('./df_customer_info_tr.pkl')

#### Gather basic information about the loan customer is applying for

In [82]:
# Informations sur la demande prêt
df_customer_loan = dash_df[['SK_ID_CURR', 'NAME_CONTRACT_TYPE',
                            'AMT_CREDIT',
                            'AMT_ANNUITY', 'AMT_GOODS_PRICE',
                            'NAME_HOUSING_TYPE']]

In [83]:
df_customer_loan = df_customer_loan.rename(columns = {
     'NAME_CONTRACT_TYPE' : 'Loan Type',
     'AMT_CREDIT' : 'Loan amount ($)',
     'AMT_ANNUITY' : 'Annuity ($)',
     'AMT_GOODS_PRICE' : 'Goods price ($)',
     'NAME_HOUSING_TYPE' : 'Housing Type'})
df_customer_loan.loc[df_customer_loan['SK_ID_CURR'] == 100271]

,SK_ID_CURR,Loan Type,Loan amount ($),Annuity ($),Goods price ($),Housing Type
36,100271,Cash Loans,1800000.0,98770.5,1800000.0,House / apartment


In [84]:
df_customer_loan.head(2000).to_pickle('./df_customer_loan.pkl')

In [85]:
data_test_ltd.to_pickle('./data_test_ltd.pkl')

In [86]:
data_train_ltd.to_pickle('./data_train_ltd.pkl')

#### Gather basic information about the loan customer is applying for

In [87]:
# Informations sur la demande prêt
df_customer_loan_tr = dashboard[['SK_ID_CURR', 'NAME_CONTRACT_TYPE',
                            'AMT_CREDIT',
                            'AMT_ANNUITY', 'AMT_GOODS_PRICE',
                            'NAME_HOUSING_TYPE']]

In [88]:
df_customer_loan_tr = df_customer_loan_tr.rename(columns = {
     'NAME_CONTRACT_TYPE' : 'Loan Type',
     'AMT_CREDIT' : 'Loan amount ($)',
     'AMT_ANNUITY' : 'Annuity ($)',
     'AMT_GOODS_PRICE' : 'Goods price ($)',
     'NAME_HOUSING_TYPE' : 'Housing Type'})
df_customer_loan_tr.loc[df_customer_loan_tr['SK_ID_CURR'] == 100002]

,SK_ID_CURR,Loan Type,Loan amount ($),Annuity ($),Goods price ($),Housing Type
0,100002,Cash Loans,406597.5,24700.5,351000.0,House / apartment


In [89]:
df_customer_loan_tr.to_pickle('./df_customer_loan_tr.pkl')

In [90]:
data_test_ltd.to_pickle('./data_test_ltd.pkl')

### Full set

In [91]:
# Copie du jeu de test
test = data_test_boruta.copy(deep=True)
# Sauvegarde des identifiants des clients inutiles aux prédictions
customers_id = test['SK_ID_CURR']
# Préparation de la matrice à soumettre aux prédictions 
X_test = test.drop('SK_ID_CURR', axis=1)

In [92]:
# Score des prédictions de probabiltés
y_proba = best_model.predict_proba(X_test)[:, 1]
# Customers predictions
# class 0 : Non defaulters, class 1 : Defaulters
y_pred = best_model.predict(X_test)
# Constitution du dataframe pour merge
df_score = pd.DataFrame({'SK_ID_CURR' : customers_id,
                         'PREDICTED_CLASS' : y_pred,
                         'CUSTOMER_SCORE' : y_proba,
                         'CUSTOMER_SCORE_%' : np.round(y_proba * 100, 1)})
df_score1 = df_score.sort_values(by='CUSTOMER_SCORE_%', ascending=False)
df_score1.head(50)

,SK_ID_CURR,PREDICTED_CLASS,CUSTOMER_SCORE,CUSTOMER_SCORE_%
333823,291592,1.0,0.802025,80.2
317153,170148,1.0,0.789027,78.9
330321,265895,1.0,0.763154,76.3
334930,300137,1.0,0.755004,75.5
307580,100568,1.0,0.749763,75.0
323571,215910,1.0,0.741322,74.1
338538,325553,1.0,0.736372,73.6
333360,288138,1.0,0.733554,73.4
313796,146090,1.0,0.728786,72.9
351538,421386,1.0,0.723730,72.4


In [93]:
df_score.loc[df_score['SK_ID_CURR'] == 100038]

,SK_ID_CURR,PREDICTED_CLASS,CUSTOMER_SCORE,CUSTOMER_SCORE_%
307510,100038,0.0,0.087572,8.8


In [94]:
df_score['PREDICTED_CLASS'].value_counts()

0.0    48606
1.0      138
Name: PREDICTED_CLASS, dtype: int64

In [96]:
# Ajout des prédictions au dataframe du jeu de test
dash_df = df_score.merge(test, on='SK_ID_CURR', how='left')
dash_df

,SK_ID_CURR,PREDICTED_CLASS,CUSTOMER_SCORE,CUSTOMER_SCORE_%,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,INS_DBD_MEAN,INS_LATE_PAYMENT_SUM,INS_36M_DAYS_ENTRY_PAYMENT_MIN,INS_36M_AMT_INSTALMENT_MEAN,INS_36M_AMT_PAYMENT_SUM,INS_36M_SIGNIFICANT_LATE_PAYMENT_MEAN,INS_60M_LATE_PAYMENT_MEAN,INS_60M_SIGNIFICANT_LATE_PAYMENT_MEAN,BUREAU_ACTIVE_CREDIT_TO_INCOME_RATIO,CURRENT_TO_APPROVED_CREDIT_MIN_RATIO
0,100001,0.0,0.011471,1.1,0,0,135000.0,568800.0,0,0,...,8.859375,4.0,-840.0,15708.870117,3.499691e+05,0.910645,1.000000,1.000000,6.546875,0.041809
1,100005,0.0,0.072649,7.3,0,0,99000.0,222768.0,0,0,...,23.671875,8.0,-736.0,6240.205078,5.616184e+04,0.888672,0.888672,0.888672,6.046875,0.180298
2,100013,0.0,0.051149,5.1,0,1,202500.0,663264.0,-1,0,...,5.722656,49.0,-2704.0,11124.233398,1.444700e+06,0.226074,0.316162,0.270996,4.632812,0.036530
3,100028,0.0,0.148063,14.8,0,0,315000.0,1575000.0,0,0,...,3.265625,55.0,-1302.0,4730.878418,3.506999e+05,0.348389,0.486816,0.486816,2.746094,0.037781
4,100038,0.0,0.087572,8.8,0,1,180000.0,625500.0,0,0,...,12.250000,12.0,-802.0,11100.337891,1.332040e+05,1.000000,1.000000,1.000000,4.632812,0.148071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,0.0,0.017779,1.8,0,0,121500.0,412560.0,0,0,...,7.332031,3.0,-661.0,91036.453125,2.731094e+05,1.000000,1.000000,1.000000,6.898438,0.617188
48740,456222,0.0,0.036961,3.7,0,0,157500.0,622413.0,0,2,...,6.000000,62.0,-815.0,10101.311523,4.803925e+05,0.836914,0.836914,0.836914,4.632812,0.043030
48741,456223,0.0,0.012649,1.3,0,1,202500.0,315000.0,0,2,...,31.625000,8.0,-818.0,23158.992188,1.852719e+05,1.000000,1.000000,1.000000,0.249634,0.353271
48742,456224,0.0,0.006910,0.7,0,0,225000.0,450000.0,1,2,...,7.277344,26.0,-559.0,23451.705078,4.455824e+05,1.000000,1.000000,1.000000,9.625000,0.089905


### Limited set

In [97]:
# Copie du jeu de test
test_ltd = data_test_ltd.copy(deep=True)
# Sauvegarde des identifiants des clients inutiles aux prédictions
customers_id_ltd = test_ltd['SK_ID_CURR']
# Préparation de la matrice à soumettre aux prédictions 
X_test_ltd = test_ltd.drop('SK_ID_CURR', axis=1)

In [126]:
# Score des prédictions de probabiltés
y_proba = best_model.predict_proba(X_test_ltd)[:, 1]
# Customers predictions
# class 0 : Non defaulters, class 1 : Defaulters
y_pred = best_model.predict(X_test_ltd)
y_pred = (y_proba > 0.1)
y_pred = np.multiply(y_pred, 1)
# Constitution du dataframe pour merge
df_score = pd.DataFrame({'SK_ID_CURR' : customers_id_ltd,
                         'PREDICTED_CLASS' : y_pred,
                         'CUSTOMER_SCORE' : y_proba,
                         'CUSTOMER_SCORE_%' : np.round(y_proba * 100, 1)})
df1 = df_score.sort_values(by='CUSTOMER_SCORE_%', ascending=False)
df1.head(130)

,SK_ID_CURR,PREDICTED_CLASS,CUSTOMER_SCORE,CUSTOMER_SCORE_%
307580,100568,1,0.749763,75.0
308504,106854,1,0.674048,67.4
308646,108045,1,0.626114,62.6
309307,112771,1,0.559066,55.9
308140,104410,1,0.556654,55.7
...,...,...,...,...
307891,102617,1,0.168785,16.9
309360,113087,1,0.167546,16.8
307721,101427,1,0.166850,16.7
308394,106116,1,0.167408,16.7


In [120]:
df_score.loc[df_score['SK_ID_CURR'] == 100038]

,SK_ID_CURR,PREDICTED_CLASS,CUSTOMER_SCORE,CUSTOMER_SCORE_%
307510,100038,0,0.087572,8.8


In [121]:
df_score['PREDICTED_CLASS'].value_counts()

0    1701
1     299
Name: PREDICTED_CLASS, dtype: int64

### Limited train set

In [44]:
# Copie du jeu de test
train_ltd = data_train_ltd.copy(deep=True)
# Sauvegarde des identifiants des clients inutiles aux prédictions
customers_id_ltd = train_ltd['SK_ID_CURR']
# Préparation de la matrice à soumettre aux prédictions 
X_train_ltd = train_ltd.drop('SK_ID_CURR', axis=1)

In [45]:
# Score des prédictions de probabiltés
y_proba = best_model.predict_proba(X_train_ltd)[:, 1]
# Customers predictions
# class 0 : Non defaulters, class 1 : Defaulters
y_pred = best_model.predict(X_train_ltd)
y_pred = (y_proba > 0.1)
y_pred = np.multiply(y_pred, 1)
# Constitution du dataframe pour merge
df_score_tr = pd.DataFrame({'SK_ID_CURR' : customers_id_ltd,
                         'PREDICTED_CLASS' : y_pred,
                         'CUSTOMER_SCORE' : y_proba,
                         'CUSTOMER_SCORE_%' : np.round(y_proba * 100, 1)})
df = df_score_tr.sort_values(by='CUSTOMER_SCORE_%', ascending=False)
df.head(500)

,SK_ID_CURR,PREDICTED_CLASS,CUSTOMER_SCORE,CUSTOMER_SCORE_%
211,100246,1,0.788699,78.9
560,100646,1,0.749247,74.9
726,100839,1,0.748287,74.8
695,100804,1,0.702962,70.3
344,100396,1,0.696607,69.7
...,...,...,...,...
998,101155,0,0.091928,9.2
1646,101931,0,0.090518,9.1
580,100666,0,0.091121,9.1
1061,101232,0,0.091178,9.1


In [46]:
df_score_tr.loc[df_score_tr['SK_ID_CURR'] == 100804]

,SK_ID_CURR,PREDICTED_CLASS,CUSTOMER_SCORE,CUSTOMER_SCORE_%
695,100804,1,0.702962,70.3


In [47]:
df_score_tr['PREDICTED_CLASS'].value_counts()

0    1548
1     452
Name: PREDICTED_CLASS, dtype: int64

In [48]:
# Test set sans l'identifiant
X_train_ltd = data_train_ltd.set_index('SK_ID_CURR')
# Entraînement de shap sur le train set
shap_explainer = shap.Explainer(best_model, X_train_ltd)
shap_values_tr_ltd = shap_explainer(X_train_ltd, check_additivity=False)

100%|===================| 1999/2000 [22:59<00:00]        

In [49]:
pickle.dump(shap_values_tr_ltd, open('shap_values_tr_ltd.pkl', 'wb'))

*************************************

In [101]:
def explain(customer_id):  
    
    explainer = shap.TreeExplainer(best_model)
    X_shap = test.set_index('SK_ID_CURR')
    X_test_cur = X_shap.loc[[customer_id]]
    X_test_cur_array = X_test_cur.values.reshape(1, -1)

    shap_values_cur = explainer.shap_values(X_test_cur_array)
    exp_val = explainer.expected_value[1]
    return exp_val

exp_val = explain(451379)
exp_val

-3.1573977065395495

In [102]:
pickle.dump(exp_val, open('exp_value.pkl', 'wb'))

### Neighbours

In [103]:
data_test_boruta = pd.read_pickle("test_data_boruta.pkl")
data_test_boruta

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,...,INS_DBD_MEAN,INS_LATE_PAYMENT_SUM,INS_36M_DAYS_ENTRY_PAYMENT_MIN,INS_36M_AMT_INSTALMENT_MEAN,INS_36M_AMT_PAYMENT_SUM,INS_36M_SIGNIFICANT_LATE_PAYMENT_MEAN,INS_60M_LATE_PAYMENT_MEAN,INS_60M_SIGNIFICANT_LATE_PAYMENT_MEAN,BUREAU_ACTIVE_CREDIT_TO_INCOME_RATIO,CURRENT_TO_APPROVED_CREDIT_MIN_RATIO
307506,100001,0,0,135000.0,568800.0,0,0,1,0,0.018845,...,8.859375,4.0,-840.0,15708.870117,3.499691e+05,0.910645,1.000000,1.000000,6.546875,0.041809
307507,100005,0,0,99000.0,222768.0,0,0,1,0,0.035797,...,23.671875,8.0,-736.0,6240.205078,5.616184e+04,0.888672,0.888672,0.888672,6.046875,0.180298
307508,100013,0,1,202500.0,663264.0,-1,0,1,0,0.019104,...,5.722656,49.0,-2704.0,11124.233398,1.444700e+06,0.226074,0.316162,0.270996,4.632812,0.036530
307509,100028,0,0,315000.0,1575000.0,0,0,1,0,0.026398,...,3.265625,55.0,-1302.0,4730.878418,3.506999e+05,0.348389,0.486816,0.486816,2.746094,0.037781
307510,100038,0,1,180000.0,625500.0,0,0,1,0,0.010033,...,12.250000,12.0,-802.0,11100.337891,1.332040e+05,1.000000,1.000000,1.000000,4.632812,0.148071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356245,456221,0,0,121500.0,412560.0,0,0,3,0,0.002043,...,7.332031,3.0,-661.0,91036.453125,2.731094e+05,1.000000,1.000000,1.000000,6.898438,0.617188
356246,456222,0,0,157500.0,622413.0,0,2,1,0,0.035797,...,6.000000,62.0,-815.0,10101.311523,4.803925e+05,0.836914,0.836914,0.836914,4.632812,0.043030
356247,456223,0,1,202500.0,315000.0,0,2,1,0,0.026398,...,31.625000,8.0,-818.0,23158.992188,1.852719e+05,1.000000,1.000000,1.000000,0.249634,0.353271
356248,456224,0,0,225000.0,450000.0,1,2,1,0,0.018845,...,7.277344,26.0,-559.0,23451.705078,4.455824e+05,1.000000,1.000000,1.000000,9.625000,0.089905


In [104]:
most_imp_feat = ['INS_60M_SIGNIFICANT_LATE_PAYMENT_MEAN',
            'EXT_SOURCES_NANMEDIAN',
            'INS_36M_AMT_PAYMENT_SUM',
            'EXT_SOURCES_MIN',
            'INS_60M_LATE_PAYMENT_MEAN',
            'EXT_SOURCES_MAX',
            'CREDIT_TO_ANNUITY_RATIO',
            'ANNUITY_TO_INCOME_RATIO',
            'BUREAU_CONSUMER_DAYS_CREDIT_ENDDATE_MAX',
            'BUREAU_ACTIVE_DAYS_CREDIT_UPDATE_MEAN',
                 
            'ID_TO_BIRTH_RATIO',
            'DAYS_BIRTH',
            'AMT_CREDIT',
            'PREV_DAYS_DECISION_MEAN',
            'INS_DBD_MEAN',
            'PREV_Consumer_AMT_CREDIT_SUM',
            'PHONE_TO_BIRTH_RATIO',
            'INCOME_TO_BIRTH_RATIO',
            'CURRENT_TO_APPROVED_CREDIT_MIN_RATIO',
            'APPROVED_CREDIT_TO_ANNUITY_RATIO_MEAN']

In [105]:
X_test = data_test_boruta[most_imp_feat]

In [55]:
X_train = data_train_boruta[most_imp_feat]

In [106]:
# Standardization for KNN - test set
scaler = StandardScaler()
X_test_std = scaler.fit_transform(X_test)
X_test_std = pd.DataFrame(X_test_std, index=X_test.index, columns=X_test.columns.to_list())
X_test_std.shape

(48744, 20)

In [57]:
# Standardization for KNN - train set
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_train_std = pd.DataFrame(X_train_std, index=X_train.index, columns=X_train.columns.to_list())
X_train_std.shape

(307506, 20)

In [107]:
dash_df.shape

(48744, 90)

In [108]:
y_test = dash_df['CUSTOMER_SCORE'].astype('int64')
y_test.shape

(48744,)

In [59]:
y_train = dash_df['CUSTOMER_SCORE'].astype('int64')
y_train.shape

(307506,)

In [109]:
# Model initialization
knn_test = KNeighborsClassifier(10, n_jobs = -1)
# Model training
knn_test.fit(X_test_std, y_test)

KNeighborsClassifier(n_jobs=-1, n_neighbors=10)

In [110]:
# Index of the 10 nearest neighbors
index_10_neigh_test = knn_test.kneighbors(X_test_std)[1]
dash_df['INDEX_10_NEIGH_MEAN'] = \
    [index for index in index_10_neigh_test]

In [111]:
# Score of the 10 nearest neighbors
dash_df['INDEX_10_NEIGH_MEAN'] = \
    [dash_df['CUSTOMER_SCORE'].iloc[index].mean()
     for index in index_10_neigh_test]

In [112]:
# dataframe reconstitution
dash_df['10_NEIGH_MEAN_SCORE'] = \
    [dash_df['CUSTOMER_SCORE'].iloc[index].mean()
     for index in index_10_neigh_test]

In [113]:
df2 = dash_df.sort_values(by='10_NEIGH_MEAN_SCORE', ascending=False)
df2.head(100)

,SK_ID_CURR,PREDICTED_CLASS,CUSTOMER_SCORE,CUSTOMER_SCORE_%,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,INS_36M_DAYS_ENTRY_PAYMENT_MIN,INS_36M_AMT_INSTALMENT_MEAN,INS_36M_AMT_PAYMENT_SUM,INS_36M_SIGNIFICANT_LATE_PAYMENT_MEAN,INS_60M_LATE_PAYMENT_MEAN,INS_60M_SIGNIFICANT_LATE_PAYMENT_MEAN,BUREAU_ACTIVE_CREDIT_TO_INCOME_RATIO,CURRENT_TO_APPROVED_CREDIT_MIN_RATIO,INDEX_10_NEIGH_MEAN,10_NEIGH_MEAN_SCORE
21153,253499,1.0,0.515717,51.6,0,1,184500.0,517500.0,0,0,...,-816.0,62932.746094,2.517310e+05,1.000000,1.000000,1.000000,7.714844,0.137085,0.420394,0.420394
31036,325590,0.0,0.398855,39.9,0,0,292500.0,601470.0,0,0,...,-1223.0,63645.203125,1.077283e+06,0.625000,0.687500,0.625000,4.632812,0.393066,0.394981,0.394981
74,100568,1.0,0.749763,75.0,0,0,112500.0,312768.0,0,0,...,-749.0,8640.897461,1.555362e+05,1.000000,1.000000,1.000000,4.960938,0.170410,0.382799,0.382799
26317,291592,1.0,0.802025,80.2,0,0,112500.0,234369.0,0,0,...,-437.0,5181.911621,8.317989e+04,0.882324,0.941406,0.882324,0.749512,0.148438,0.382691,0.382691
25093,282773,0.0,0.435266,43.5,0,0,135000.0,573408.0,0,0,...,-269.0,6100.830078,2.440332e+04,1.000000,1.000000,1.000000,19.703125,0.040680,0.381508,0.381508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33023,340610,1.0,0.612609,61.3,0,1,90000.0,552555.0,0,0,...,-532.0,7289.818848,2.624335e+05,0.416748,0.416748,0.416748,1.250000,0.028641,0.281482,0.281482
4543,132485,0.0,0.141607,14.2,0,1,135000.0,594121.5,0,0,...,-2912.0,2306.529785,2.861909e+05,0.291992,0.291992,0.291992,4.632812,0.064880,0.281144,0.281144
13449,197529,0.0,0.329246,32.9,0,1,90000.0,358344.0,0,1,...,-739.0,11224.202148,1.010089e+06,0.539551,0.565918,0.539551,5.800781,0.047333,0.281030,0.281030
33544,344762,0.0,0.144531,14.5,0,1,76500.0,315000.0,0,0,...,-992.0,12282.608398,1.171168e+05,0.000000,0.500000,0.227295,4.632812,0.122009,0.280905,0.280905


In [116]:
# customer score 
customer_id = 344762
customer_score = int(np.rint(dash_df[dash_df['SK_ID_CURR'] == customer_id]['CUSTOMER_SCORE_%']))
customer_score

14

In [117]:
# Score du client en pourcentage?
customer_id = 344762
mean_neigh_score = int(np.rint(dash_df[dash_df['SK_ID_CURR'] == customer_id]['10_NEIGH_MEAN_SCORE'] * 100))
mean_neigh_score

28

In [118]:
pickle.dump(dash_df, open('dash_df.pkl', 'wb'))